In [1]:
%matplotlib inline

import boto3
import time
import json
import os

import numpy as np
import pandas as pd
import math

from datetime import datetime, timezone
from datetime import timedelta as td

session = boto3.client('logs')

/Users/nikita/Code/Thesis/ve/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# __file__ does not exist while executing a notebok from jupyter but does exist when importing it
# this ensures __file__ always exists
__file__ = globals().get('__file__', os.path.join(os.path.abspath("./results.ipynb"), ))

In [3]:
def download_results():
    start = int(datetime(2021, 5, 1).timestamp())
    results = []

    while True:
        print('Querying')

        # Query for the first time beginning in 1970
        query = session.start_query(
            logGroupName='/aws/batch/job',
            startTime=start,
            endTime=int(time.time()),
            queryString="""
                fields @message, @timestamp
                | filter success == 1
                | sort @timestamp asc
                | limit 10000
            """,
            limit=10000
        )

        # Loop untill the query response is ready
        query_result = {'status': 'Scheduled'}
        while query_result['status'] in ('Scheduled', 'Running'):
            query_result = session.get_query_results(queryId=query['queryId'])
            time.sleep(1)

        tmp = query_result

        # If we reached the end, we don't have any more data, we break
        if len(query_result['results']) == 0:
            break

        # If we did not reach the end, we get the timestamp of the last line and use this
        # as a starting point for the next iteration
        assert query_result['results'][-1][1]['field'] == "@timestamp"
        start = int(datetime.strptime(query_result['results'][-1][1]['value'], "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo=timezone.utc).timestamp()) + 1

        # Process the data a little bit and push it to results
        data = [
            json.loads(row[0]['value'])
            for row in query_result['results']
        ]
        results += data

    print(f"Done, with {len(results)} results")
    
    return results

In [4]:
SOURCE_DIR = os.path.dirname(os.path.dirname(__file__))


DATA_DIR = os.path.join(SOURCE_DIR, "data/")
CLOUDWATCH_DATA_PATH = os.path.join(DATA_DIR, "cloudwtach-results.jsonl")

In [5]:
def write_results(results):
    with open(CLOUDWATCH_DATA_PATH, "w") as fd:
        for row in results:
            json.dump(row, fd)
            fd.write("\n")

In [6]:
write_results(download_results())

Querying
Querying
Done, with 2987 results


In [7]:
import glob

def read_results():
    with open(CLOUDWATCH_DATA_PATH) as fd:
        cloudwatch = [json.loads(l) for l in fd]

        local = []

        for path in glob.glob(os.path.join(DATA_DIR, "local-results", "*.jsonl")):
            with open(path) as fd:
                data = [json.loads(l) for l in fd if l.startswith("{")]
            local += data

    raw_results = cloudwatch + local

    for row in raw_results:
        if row['winner'] is None:
            row['winner'] = 0.5

    df = pd.DataFrame(raw_results)
    return df

In [8]:
def get_locals():
    class Player:
        def __init__(self, *args, **kwargs):
            self.args = args
            self.kwargs = kwargs

    class GreedyPlayer(Player):
        pass

    class MCTSPlayer(Player):
        pass

    class UCTPlayer(Player):
        pass


    class GreedyUCTPlayer(Player):
        pass


    class RandomPlayer(Player):
        pass
    return locals()

In [9]:
player_locals = get_locals()

In [10]:
def process_results(df):
    

    # Fix bad data
    df['version'] = df.version.fillna(1).astype(int)
    df['side'] = df['side'].fillna(-1).astype(int)

    # Only keep usable data
    df = df[df.version >= 2]

    is_normalized = df['side'] == 0

    to_normalise = df[~is_normalized].copy()

    # Swap columns
    opponents = to_normalise['opponent'].copy()
    players = to_normalise['player'].copy()

    to_normalise['player'] = opponents
    to_normalise['opponent'] = players

    # Swap winner
    to_normalise['winner'] = 1 - to_normalise['winner']

    # Swap individual scores
    to_normalise['score'] = to_normalise['score'].map(lambda x: list(reversed(x)))

    # Concatenate the 2 sides
    normalised = pd.concat([
        df[is_normalized].copy(),
        to_normalise.copy()
    ]).copy()


    # Evaluate both players
    normalised['player_eval'] = normalised['player'].map(lambda x: eval(x, globals(), player_locals))
    normalised['opponent_eval'] = normalised['opponent'].map(lambda x: eval(x, globals(), player_locals))
    
    return normalised

In [11]:
raw_results = read_results()

In [12]:
results = process_results(raw_results)